In [1]:
import tf_keras
import tf.records as tfr
import tf.ctc as ctc
import moveread_ocr as mo

2024-06-10 20:20:46.786013: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 20:20:46.793072: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 20:20:46.959238: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 20:20:51.062342: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
char2num = tf_keras.layers.StringLookup(vocabulary=list(mo.VOCABULARY))
num2char = tf_keras.layers.StringLookup(vocabulary=char2num.get_vocabulary(), invert=True)

2024-06-10 20:20:54.226835: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-10 20:20:54.508147: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
model = tf_keras.models.load_model('ocr_model.h5')

In [4]:
ds = tfr.batched_read(mo.records.SCHEMA, ['data/tfrecords/0.tfrecord.gz'], compression='GZIP')

In [5]:
for x in ds:
  break

In [8]:
z = model(x['image'])

2024-06-10 20:21:36.265570: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.
2024-06-10 20:21:36.700851: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.
2024-06-10 20:21:37.257712: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.
2024-06-10 20:21:37.407255: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33554432 exceeds 10% of free system memory.
2024-06-10 20:21:37.453343: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


In [9]:
preds, probs = ctc.beam_decode(z)

In [12]:
preds

[SparseTensor(indices=tf.Tensor(
 [[ 0  0]
  [ 0  1]
  [ 0  2]
  ...
  [31 24]
  [31 25]
  [31 26]], shape=(756, 2), dtype=int64), values=tf.Tensor(
 [30  7 30 13 30  7 30 16 30 11 30 16 30 11 30 27 30 27 30 11 30 27 30 30
   7 30  7 30 17 30 18 30 22 30 22 30 27 30 27 30 27 30 27 30 27 30 30  8
  30  7 30 14 30 17 30 16 30 16 30 10 30 30  6 30  3 30 20 30  7 30 21 30
  21 30 27 30 13 30  1 30  2 30 18 30 27 30 30  5 30 10 30  7 30  4 30 11
  30 11 30  3 30  8 30 14 30 29 30  2 30 27 30 27 30 30  7 30  3 30 10 30
  23 30  9 30 14 30 27 30 27 30  7 30  7 30 27 30 30  3 30 23 30 20 30  7
  30 15 30 11 30 11 30 16 30  7 30 13 30 14 30  6 30  3 30 30  7 30  9 30
   8 30 17 30  9 30 23 30  7 30  2 30 27 30 27 30 30  7 30  7 30 14 30 15
  30 11 30 23 30 20 30 20 30 11 30 27 30 27 30 27 30 30  3 30  7 30  7 30
  24 30  7 30 17 30 27 30 27 30 27 30 27 30 30 14 30  7 30 17 30 17 30 11
  30 21 30  3 30 22 30  7 30 22 30 27 30 27 30 27 30 30 15 30 15 30 10 30
   7 30 15 30  2 30 16 30 11 30 17 30

In [10]:
mo.stringify_labels(preds[0], num2char)

['DgDmDgDpDkDpDkDADADkDAD',
 'DgDgDqDrDvDvDADADADADAD',
 'DhDgDnDqDpDpDjD',
 'DfDcDtDgDuDuDADmDaDbDrDAD',
 'DeDjDgDdDkDkDcDhDnDCDbDADAD',
 'DgDcDjDwDiDnDADADgDgDAD',
 'DcDwDtDgDoDkDkDpDgDmDnDfDcD',
 'DgDiDhDqDiDwDgDbDADAD',
 'DgDgDnDoDkDwDtDtDkDADADAD',
 'DcDgDgDxDgDqDADADADAD',
 'DnDgDqDqDkDuDcDvDgDvDADADAD',
 'DoDoDjDgDoDbDpDkDqDqDpDADAD',
 'DgDgDgDmD',
 'DBDBDgDwDwDpDgDADADADAD',
 'DgDgDqDqDqDwDwDnDADADcD',
 'DnDnDgDpDgDBDADADADADAD',
 'DgDgDgDoDkDADADADADAD',
 'DeDqDxDcDkDtDgDgDxDtDqDtDADAD',
 'DnDoDeDeDgDgDkDkDADADADAD',
 'DgDiDgDcDpDvDADADADADADA',
 'DoDwDoDkDuDxDnDxDbDwDnDkDcDgDAD',
 'DgDgDqDpDiDpDgDvDADADAD',
 'DfDfDkDeDkDoDnDkDqDbDkDhDhDgDBDeDAD',
 'DgDgDpDkDqDkDADADA',
 'DgDgDcDtDcDADgDcDcDADADAD',
 'DnDcDxDcDkDqDoDADADAD',
 'DgDoDrDvDuDADvDADADADAD',
 'DgDgDgDoDpDvDADADADAD',
 'DeDeDiDqDfDkDmDkDsDjDADADlD',
 'DnDcDoDcDeDqDpDkDyDiDbDmD',
 'DgDgDkDeDpDADADgDAD',
 'DgDtDqDpDgDjDADgDbDjDfDeDAD']